In [ ]:
!pip install pyshp fiona

     |████████████████████████████████| 219 kB 5.0 MB/s 
     |████████████████████████████████| 15.4 MB 35 kB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37324 sha256=a75a1a29bf1e01179b34fb5f290c159a369aec5f609c165c084bcc67a351d5e4
  Stored in directory: /root/.cache/pip/wheels/43/f8/87/53c8cd41545ba20e536ea29a8fcb5431b5f477ca50d5dffbbe
Successfully built pyshp


In [ ]:
import numpy as np
import pandas as pd
import datetime
import math
import shapefile
from shapely import geometry
import fiona
from collections import defaultdict
from google.colab import drive
from datetime import date, timedelta

drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'sustainlab'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

In [ ]:
# Import the Earth Engine Python Package
import ee

# Initialize the Earth Engine object, using the authentication credentials.
ee.Authenticate()
ee.Initialize()

In [ ]:
sentinel = ee.ImageCollection("COPERNICUS/S2").select(['B4', 'B3', 'B2'])
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(['SR_B4', 'SR_B3', 'SR_B2'])

In [ ]:
fname = 'hold'
device = 'sentinel'
with fiona.open(f'shapefiles/{fname}.shp') as shapefile:
    # geoms = [feature['geometry'] for feature in shapefile]
    # ids = [feature['properties']['id'] for feature in shapefile]
    # dates = [feature['properties']['date'] for feature in shapefile]
    i = 0
    for j in range(2900*0, len(shapefile)):
        feature = shapefile[j]
        i += 1
        geom = feature['geometry']
        geom = geom['coordinates'][0][:4]
        xmin = geom[0][0]
        ymin = geom[0][1]
        xmax = geom[2][0]
        ymax = geom[2][1]
        geom = ee.Geometry.Rectangle([ymin, xmin, ymax, xmax])
        # print([ymin, xmin, ymax, xmax])
        id = feature['properties']['id']
        capturedate = date.fromisoformat(feature['properties']['date'])
        delta = timedelta(days=+35)
        startdate = str(capturedate - delta)
        enddate = str(capturedate + delta)
        # print(id, capturedate)
        # print(startdate, enddate)
        img = None
        if device == 'sentinel':
            s2 = sentinel.filter(ee.Filter.date(startdate, enddate)).filterBounds(geom).sort('CLOUDY_PIXEL_PERCENTAGE').first()
            img = s2.visualize(min=[0.0, 0.0, 0.0], max=[5000, 5000, 5000])
        else:
            s2 = landsat.filter(ee.Filter.date(startdate, enddate)).filterBounds(geom).sort('CLOUD_COVER_LAND').first()
            img = s2.visualize(min=[0.0, 0.0, 0.0], max=[30000.0, 30000.0, 30000.0])
        config = {
            'folder':f'xview{device}{fname}',
            'scale': 10,
            'region': geom
        }
        task = ee.batch.Export.image.toDrive(img, id, **config)
        task.start()
        if i%100 == 0:
            print(i)
        if i == 2900:
            print('start next from j=',j,'+1')
            break
    print(i, 'done')

In [ ]:
while task.status()['state'] in ['READY', 'RUNNING']:
  print(task.status())
  time.sleep(10)
else:
  print(task.status())

In [ ]:
!earthengine task list

In [ ]:
!earthengine task cancel all